In [1]:
import numpy as np
import pandas as pd
import sys
import os 

from os import path
from eppy import eppy
from eppy import hvacbuilder
from eppy import modeleditor
from eppy.modeleditor import IDF

In [2]:
def eppy_connection(idffile):
    fp = open(idffile)
    lines=fp.readlines()
    vers = ""
    vers = vers.join(lines[2:12])
    vers
    if vers.find('8.7') != -1:
        print('Using EnergyPlus version 8.7')
        iddfile = 'C:\\EnergyPlusV8-7-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.8') != -1:
        print('Using EnergyPlus version 8.8')
        iddfile = 'C:\\EnergyPlusV8-8-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.9') != -1:
        print('Using EnergyPlus version 8.9')
        iddfile = 'C:\\EnergyPlusV8-9-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('9.0') != -1:
        print('Using EnergyPlus version 9.0.1')
        iddfile = 'C:\\EnergyPlusV9-0-1\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    else:
        print('IDF Type not found on line 3')

In [3]:
#define paths
commonSlab_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\jEplus\\idfs\\common_hp_slab_IECC_2012_V8.7.idf")
commonBsmt_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\jEplus\\idfs\\common_hp_heatedbsmtIECC_2012_V8.7.idf")
gardenSlab_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\jEplus\\idfs\\garden_hp_slab_IECC_2012_V8.7.idf")
gardenBsmt_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\jEplus\\idfs\\garden_hp_heatedbsmtIECC_2012_V8.7.idf")

#make connections
commonSlab = eppy_connection(commonSlab_path)
commonBsmt = eppy_connection(commonBsmt_path)
gardenSlab = eppy_connection(gardenSlab_path)
gardenBsmt = eppy_connection(gardenBsmt_path)

# create groups
idfs = [commonSlab, commonBsmt, gardenSlab, gardenBsmt]
common = [commonSlab, commonBsmt]
garden = [gardenSlab, gardenBsmt]
bsmt = [commonBsmt, gardenBsmt]
slab = [commonSlab, gardenSlab]

Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7


In [4]:
# variables
siz_flowrate = 0.01
siz_method = 'Flow/System'
cool_cap = 1000
heat_cap = 15000
pressure = 10

In [5]:
for i in range(0,len(bsmt)):
    
    # edit Sizing:System
    siz_systems = bsmt[i].idfobjects['Sizing:System']
    siz_system = [x for x in siz_systems if x.AirLoop_Name == 'Central System_unitheatedbsmt'][0]

    siz_system.Design_Outdoor_Air_Flow_Rate = siz_flowrate
    siz_system.Cooling_Supply_Air_Flow_Rate_Method =siz_method
    siz_system.Cooling_Supply_Air_Flow_Rate = siz_flowrate
    siz_system.Heating_Supply_Air_Flow_Rate_Method = siz_method
    siz_system.Heating_Supply_Air_Flow_Rate = siz_flowrate
    siz_system.Cooling_Design_Capacity = cool_cap
    siz_system.Heating_Design_Capacity = heat_cap
    
    # edit Fan:OnOff
    fans = bsmt[i].idfobjects['Fan:OnOff']
    fan = [x for x in fans if x.Name == 'Supply Fan_unitheatedbsmt'][0]

    fan.Maximum_Flow_Rate = siz_flowrate
    fan.Pressure_Rise = pressure
    
    #edit Coil:Cooling:DX:SingleSpeed
    coolings = bsmt[i].idfobjects['Coil:Cooling:DX:SingleSpeed']
    cooling = [x for x in coolings if x.Name == 'DX Cooling Coil_unitheatedbsmt'][0]

    cooling.Gross_Rated_Total_Cooling_Capacity = cool_cap
    cooling.Rated_Air_Flow_Rate = siz_flowrate 
    
    #edit Coil:Cooling:DX:SingleSpeed
    heatings = bsmt[i].idfobjects['Coil:Heating:DX:SingleSpeed']
    heating = [x for x in heatings if x.Name == 'Main DX Heating Coil_unitheatedbsmt'][0]

    heating.Gross_Rated_Heating_Capacity = heat_cap
    heating.Rated_Air_Flow_Rate = siz_flowrate

    #edit AirLoopHVAC
    airloops = bsmt[i].idfobjects['AirLoopHVAC']
    airloop = [x for x in airloops if x.Name == 'Central System_unitheatedbsmt'][0]

    airloop.Design_Supply_Air_Flow_Rate = siz_flowrate
    
    bsmt[i].save()

In [6]:
[x for x in coolings if x.Name == 'DX Cooling Coil_unitheatedbsmt'][0]


Coil:Cooling:DX:SingleSpeed,
    DX Cooling Coil_unitheatedbsmt,    !- Name
    always_avail,             !- Availability Schedule Name
    1000,                     !- Gross Rated Total Cooling Capacity
    autosize,                 !- Gross Rated Sensible Heat Ratio
    @@BSMTCLCOP@@,            !- Gross Rated Cooling COP
    0.01,                     !- Rated Air Flow Rate
    ,                         !- Rated Evaporator Fan Power Per Volume Flow Rate
    Cooling Coil Air Inlet Node_unitheatedbsmt,    !- Air Inlet Node Name
    Heating Coil Air Inlet Node_unitheatedbsmt,    !- Air Outlet Node Name
    @@BSMTCLCURVE_CAPFT@@,    !- Total Cooling Capacity Function of Temperature Curve Name
    @@BSMTCLCURVE_CAPFFF@@,    !- Total Cooling Capacity Function of Flow Fraction Curve Name
    @@BSMTCLCURVE_EIRFT@@,    !- Energy Input Ratio Function of Temperature Curve Name
    @@BSMTCLCURVE_EIRFFF@@,    !- Energy Input Ratio Function of Flow Fraction Curve Name
    @@BSMTCLCURVE_PLFFPLR@@;